In [1]:
%%writefile CMakeLists.txt
cmake_minimum_required(VERSION 3.5)
project(CudaOpenCVProject LANGUAGES CXX CUDA)

# Find OpenCV
find_package (OpenCV 4.0.0 REQUIRED)
include_directories ("/usr/include/opencv4/")

# Set CUDA architecture (change according to your GPU architecture)
set(CMAKE_CUDA_FLAGS ${CMAKE_CUDA_FLAGS} -arch=sm_75)

# Specify include directories
include_directories(${OpenCV_INCLUDE_DIRS})

# Add CUDA executable with explicitly specifying source file
add_executable(CudaImage CudaImage.cu)

# Set CUDA architectures property for the target (replace with your GPU's compute capability)
set_target_properties(CudaImage PROPERTIES CUDA_ARCHITECTURES 75)

# Link OpenCV libraries
target_link_libraries(CudaImage ${OpenCV_LIBS})

Writing CMakeLists.txt


In [2]:
%%writefile CudaImage.cu
#include </usr/include/opencv4/opencv2/opencv.hpp>
#include <iostream>
#include <chrono>
#include <filesystem>
#include <boost/filesystem.hpp>


int main() {
    // Image Path
    std::string imagePath = "/content/SampleImage.jpeg"; // Replace with your uploaded image path
    cv::Mat image = cv::imread(imagePath, cv::IMREAD_GRAYSCALE);
    if (image.empty()) {
        std::cerr << "OpenCV version: " << CV_VERSION << std::endl;
        std::cerr << "Image load failed!" << std::endl;
        return -1;
    }

    cv::Mat blurredImageCPU(image.size(), image.type());

    // CPU Gaussian Blur
    auto startCPU = std::chrono::high_resolution_clock::now();
    cv::GaussianBlur(image, blurredImageCPU, cv::Size(3, 3), 3.0);
    auto endCPU = std::chrono::high_resolution_clock::now();

    // Calculate execution times
    auto cpuDuration = std::chrono::duration<double, std::milli>(endCPU - startCPU).count();
    std::cout << "CPU Time: " << cpuDuration << " ms" << std::endl;

    // Get the extension of the input image
    std::filesystem::path p(imagePath);
    std::string extension = imagePath.substr(imagePath.find_last_of("."));

    // Save ONLY blurred images
    cv::imwrite("/content/cpu_blurred_image" + extension, blurredImageCPU);

    // Display confirmation message
    std::cout << "Blurred image saved as: \n"
              << " - cpu_blurred_image" + extension + "\n";

    return 0;
}


Overwriting CudaImage.cu


In [3]:
!cmake .
!make .

-- The CXX compiler identification is GNU 11.4.0
-- The CUDA compiler identification is NVIDIA 12.2.140
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Found OpenCV: /usr (found suitable version "4.5.4", minimum required is "4.0.0") 
-- Configuring done (4.5s)
-- Generating done (0.0s)
-- Build files have been written to: /content


In [4]:
!nvcc `pkg-config --cflags --libs opencv4` CudaImage.cu -o CudaImage -I/usr/include/opencv4

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [5]:
!./CudaImage

CPU Time: 30.636 ms
Blurred image saved as: 
 - cpu_blurred_image.jpeg
